## Bug found on the paper CS-MIA

### The problem
<img src="figure.png" alt="figure" width="700"/>

Suppose we run 3 rounds of federated training, so we will have three shadow models, $M_s^1, M_s^2, M_s^3$, and three target models $M_t^1, M_t^2, M_t^3$. For simplicity, let's assume the shadow dataset consists of two training inputs $x_{\text{in}}^1, x_{\text{in}}^2$, and we take two other inputs not used for training, $x_{\text{out}}^1, x_{\text{out}}^2$. In this case, following the figure above, the training dataset for the attack model will be as follows:

| Conf 1| Conf 2 | Conf 3 | Label |
|-------------------|-------------------|-------------------|-------------------|
| $M_s^1(x_{\text{in}}^1)$ | $M_s^2(x_{\text{in}}^1)$ | $M_s^3(x_{\text{in}}^1)$ | 1 |
| $M_s^1(x_{\text{in}}^2)$ | $M_s^2(x_{\text{in}}^2)$ | $M_s^3(x_{\text{in}}^2)$ | 1 |
| $M_s^1(x_{\text{out}}^1)$ | $M_s^2(x_{\text{out}}^1)$ | $M_s^3(x_{\text{out}}^1)$ | 0 |
| $M_s^1(x_{\text{out}}^2)$ | $M_s^2(x_{\text{out}}^2)$ | $M_s^3(x_{\text{out}}^2)$ | 0 |

where $M_s^i(x_{in}^j)$ is the confidence of the prediction of the model $M_s^i$ over the input $x_{in}^j$.

In the code provided by the paper's authors, they build this dataset by columns. Since each column corresponds to a model, they use a loop that iterates over the models. However, in each iteration, the randomness of PyTorch's DataLoader causes the order in which they receive inputs to be different. For example, they might receive the list $(x_{\text{in}}^1, x_{\text{in}}^2, x_{\text{out}}^1, x_{\text{out}}^2)$ in the first iteration, but receive the list $(x_{\text{in}}^2, x_{\text{in}}^1, x_{\text{out}}^1, x_{\text{out}}^2)$ in the second iteration, resulting in the first two columns being in the form

| Conf 1| Conf 2 |
|-------------------|-------------------|
| $M_s^1(x_{\text{in}}^1)$ | $M_s^2(x_{\text{in}}^2)$ | 
| $M_s^1(x_{\text{in}}^2)$ | $M_s^2(x_{\text{in}}^1)$ |
| $M_s^1(x_{\text{out}}^1)$ | $M_s^2(x_{\text{out}}^1)$ | 
| $M_s^1(x_{\text{out}}^2)$ | $M_s^2(x_{\text{out}}^2)$ |

which has no sense. The same is happening when they build the dataset for the inference phase.

### Experiment

> **Note**: due to the randomness of the DataLoader you may obtain different results, but there will be the same problem.

For simplicity, we will take 10 rows from the MNIST dataset, and will use 5 as members and 5 as non-members.

In [55]:
import numpy as np
import torch
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor
from torch.utils.data import Subset, DataLoader

In [54]:
total_dataset = MNIST(root = "data/", transform = ToTensor(), download = True)
members_dataset = Subset(total_dataset, np.random.choice(np.arange(1, 1000), 5, replace = False))
non_members_dataset = Subset(total_dataset, np.random.choice(np.arange(2000, 3000), 5, replace = False))

Now we create 3 toy models.

In [63]:
class ToyNN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.layer = torch.nn.Linear(28 * 28, 64)
        self.classifier = torch.nn.Linear(64, 10)

    def forward(self, x):
        x = x.view(-1, 28 * 28)
        x = self.layer(x)
        x = self.classifier(x)
        return x
    
model1 = ToyNN()
model2 = ToyNN()
model3 = ToyNN()

We define the functions used by the authors to create the attack training dataset, but we print the batch targets in each iteration over the DataLoader, in order to check whether we are receiving the same inputs.

In [98]:
def cal_conf_assurance(model, data_loader):
    final_result = []
    with torch.no_grad():
        for batch_index, (inputs, targets) in enumerate(data_loader):
            print(f"Batch targets: {targets.squeeze().tolist()}")
            predictions = model(inputs)
            # log_softmax = nn.LogSoftmax(dim=1)
            log_softmax = torch.nn.Softmax(dim=1)
            predictions = log_softmax(predictions)
            result = torch.zeros(predictions.shape[0], dtype=torch.float32)
            for index, temp in enumerate(predictions):
                result[index] = temp[targets[index]]
            final_result += result.numpy().tolist()
    return torch.Tensor(final_result)

def get_dataset(models, train_loader, test_loader):
        # decide how to compute prediction confidence
        attack_train_x = []
        cal_assurance_function = cal_conf_assurance

        # compute confidence series of members
        print("Members:\n")
        i = 0
        for model in models:
            i += 1
            print("-"*10 + f"Confidences for model {i}")
            if isinstance(model, dict):
                model = model['model']
            assurance = cal_assurance_function(model, train_loader)
            attack_train_x.append(assurance.cpu().detach().numpy().tolist())
        attack_train_x = torch.tensor(attack_train_x).t()
        # assign confidence series of members with label 1
        attack_train_y = torch.ones(attack_train_x.shape[0], dtype=torch.long)

        # compute confidence series of non-members
        attack_test_x = []
        print("\n\nNon-members:\n")
        i = 0
        for model in models:
            i += 1
            print("-"*10 + f"Confidences for model {i}")
            if isinstance(model, dict):
                model = model['model']
            assurance = cal_assurance_function(model, test_loader)
            attack_test_x.append(assurance.cpu().detach().numpy().tolist())
        attack_test_x = torch.tensor(attack_test_x).t()
        # assign confidence series of non-members with label 0
        attack_test_y = torch.zeros(attack_test_x.shape[0], dtype=torch.long)

        # combine members and non-members as dataset for attack model
        x = torch.cat((attack_train_x, attack_test_x), 0)
        y = torch.cat((attack_train_y, attack_test_y), 0)
        return x, y

Create the dataset:

In [104]:
models = [model1, model2, model3]
train_loader = DataLoader(members_dataset, batch_size = 3, shuffle = True)
test_loader = DataLoader(non_members_dataset, batch_size = 3, shuffle = True)
attack_x, attack_y = get_dataset(models, train_loader, test_loader)
print("\n\nDataset:")
torch.cat([attack_x, attack_y.view(10,1)], 1)

Members:

----------Confidences for model 1
Batch targets: [7, 7, 3]
Batch targets: [0, 0]
----------Confidences for model 2
Batch targets: [0, 7, 0]
Batch targets: [3, 7]
----------Confidences for model 3
Batch targets: [7, 0, 0]
Batch targets: [3, 7]


Non-members:

----------Confidences for model 1
Batch targets: [8, 6, 9]
Batch targets: [7, 3]
----------Confidences for model 2
Batch targets: [9, 7, 3]
Batch targets: [8, 6]
----------Confidences for model 3
Batch targets: [7, 8, 9]
Batch targets: [3, 6]


Dataset:


tensor([[0.0724, 0.0974, 0.1017, 1.0000],
        [0.0826, 0.0910, 0.1153, 1.0000],
        [0.0965, 0.1308, 0.1168, 1.0000],
        [0.0871, 0.0846, 0.1000, 1.0000],
        [0.1039, 0.0959, 0.1149, 1.0000],
        [0.1042, 0.1193, 0.1133, 0.0000],
        [0.1077, 0.0866, 0.0936, 0.0000],
        [0.1000, 0.0949, 0.1084, 0.0000],
        [0.0693, 0.0825, 0.1181, 0.0000],
        [0.1072, 0.1167, 0.0944, 0.0000]])

As you can see, the batches are not the same for each model. Let's now take one input and calculate its confidence series by hand, so that we can check it.

In [105]:
input, target = members_dataset[3]
i = 0
softmax = torch.nn.Softmax(dim = 1)
for model in models:
    i += 1
    pred = softmax(model(input)).squeeze().tolist()
    print(f"Conf {i}: {pred[target]:.4f}")
    print("-" * 10)

Conf 1: 0.0724
----------
Conf 2: 0.0910
----------
Conf 3: 0.1149
----------


As we expected, the confidences of this input are not in the same row of the dataset. In fact, conf 1 is in the first row, conf 2 is in the second row and conf 3 is in the fifth row.